# Segmentation Testing with Guided K-Means

In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

PROJECT_ROOT = Path("..").resolve()

DATA_DIR = PROJECT_ROOT / "data" / "Converted Images"
ANNOT_DIR = PROJECT_ROOT / "data" / "Annotated Files"
OUT_DIR = PROJECT_ROOT / "outputs"
SAMPLE_DIR = OUT_DIR / "samples" / "segmentation_guided"
SAMPLE_DIR.mkdir(parents=True, exist_ok=True)

INDEX_CSV = OUT_DIR / "preprocessed_index.csv"

EXTS = {".jpg", ".jpeg", ".png", ".bmp"}
FIXED_SIZE = (300, 300)
RANDOM_STATE = 42
KMEANS_K = 3
PAD_RATIO = 0.35

def read_bgr_300(path):
    bgr = cv2.imread(str(path))
    return cv2.resize(bgr, FIXED_SIZE, interpolation=cv2.INTER_LINEAR) if bgr is not None else None

def read_gray_300(path):
    gray = cv2.imread(str(path), cv2.IMREAD_GRAYSCALE)
    return cv2.resize(gray, FIXED_SIZE, interpolation=cv2.INTER_LINEAR) if gray is not None else None

# 1. Verify Annotation Format

In [ ]:
txt_path = next(ANNOT_DIR.rglob("*.txt"))
print(f"Sample annotation file: {txt_path.name}")

with open(txt_path) as f:
    line = f.readline().strip()
print(f"Sample line: {line}")

parts = line.split()
values = list(map(float, parts))
print(f"Parsed values: {values}")

if len(values) == 5 and all(0.0 <= v <= 1.0 for v in values[1:]):
    print("✓ Format: YOLO normalized coordinates (0-1 range)")
else:
    print("⚠ Format: Possibly pixel coordinates or other format")

Sample txt: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\data\Annotated Files\Anthracnose (1).txt
First line: 1 0.456067 0.766338 0.324639 0.331689
Parsed: [1.0, 0.456067, 0.766338, 0.324639, 0.331689]
=> Format: YOLO normalized (aman untuk resize berapapun)


# 2. Load Dataset

In [ ]:
if not INDEX_CSV.exists():
    raise FileNotFoundError(f"Index CSV not found: {INDEX_CSV}")

df = pd.read_csv(INDEX_CSV)

if "orig_path" not in df.columns:
    if "path" in df.columns:
        df["orig_path"] = df["path"]
    else:
        raise ValueError(f"orig_path column not found. Available: {df.columns.tolist()}")

if "class" not in df.columns:
    df["class"] = df.get("ClassName", "Unknown")

if "Output" not in df.columns:
    df["Output"] = df["class"].astype(str).str.lower().apply(lambda x: 0 if x == "healthy" else 1)

print(f"Loaded {len(df)} images from {INDEX_CSV}")
print(f"\nClass distribution:")
print(df["class"].value_counts())
print(f"\nBinary labels (Output):")
print(df["Output"].value_counts())

Loaded: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\preprocessed_index.csv
Rows: 724
Counts per class:


class
Healthy            136
Soft_Rot           129
Gray_Blight        119
Brown_Stem_Spot    119
Anthracnose        118
Stem_Canker        103
Name: count, dtype: int64


Binary counts (Output):


Output
1    588
0    136
Name: count, dtype: int64

,orig_path,prep_path,class,Output,width,height,resize_mode,gaussian,gamma,equalization
0,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
1,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
2,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
3,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq
4,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,E:\Kuliah\Pengenalan Pola\addressing_agricultu...,Anthracnose,1,300,300,fixed300_bilinear,"(5, 5)_sigma0",1.2,hist_eq


# 3. Segmentation Utilities

In [ ]:
def kmeans_labels_lab(bgr, k=KMEANS_K):
    lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
    H, W = lab.shape[:2]
    X = lab.reshape(-1, 3).astype(np.float32)
    km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init="auto")
    return km.fit_predict(X).reshape(H, W)

def build_annot_index(annot_root: Path):
    idx = {}
    if annot_root.exists():
        for p in annot_root.rglob("*.txt"):
            idx[p.stem] = p
    return idx

ANNOT_INDEX = build_annot_index(ANNOT_DIR)
print(f"Found {len(ANNOT_INDEX)} annotation files")

def find_annotation_txt(img_path: str):
    stem = Path(img_path).stem
    return ANNOT_INDEX.get(stem)

def parse_yolo_txt(txt_path: Path):
    boxes = []
    with open(txt_path) as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split()
                if len(parts) == 5:
                    cls, xc, yc, bw, bh = int(float(parts[0])), *map(float, parts[1:])
                    boxes.append((cls, xc, yc, bw, bh))
    return boxes

def yolo_to_xyxy(box, W, H):
    _, xc, yc, bw, bh = box
    x1 = int(max(0, round((xc - bw/2) * W)))
    y1 = int(max(0, round((yc - bh/2) * H)))
    x2 = int(min(W-1, round((xc + bw/2) * W)))
    y2 = int(min(H-1, round((yc + bh/2) * H)))
    return x1, y1, x2, y2

def refine_roi_mask(mask01):
    mask = (mask01 > 0).astype(np.uint8)
    kernel = np.ones((7, 7), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    n, cc, stats, _ = cv2.connectedComponentsWithStats(mask * 255, connectivity=8)
    if n <= 1:
        return mask
    areas = stats[1:, cv2.CC_STAT_AREA]
    idx = 1 + int(np.argmax(areas))
    return (cc == idx).astype(np.uint8)

def lesion_mask_from_txt(shape_hw, txt_path: Path):
    H, W = shape_hw
    mask = np.zeros((H, W), dtype=np.uint8)
    for box in parse_yolo_txt(txt_path):
        x1, y1, x2, y2 = yolo_to_xyxy(box, W, H)
        mask[y1:y2+1, x1:x2+1] = 1
    return mask

def bbox_from_mask(mask01: np.ndarray):
    ys, xs = np.where(mask01 > 0)
    if xs.size == 0:
        return None
    return int(xs.min()), int(ys.min()), int(xs.max()), int(ys.max())

def clip_roi_to_context(roi_mask, lesion_mask, pad_ratio=PAD_RATIO):
    bb = bbox_from_mask(lesion_mask)
    if bb is None:
        return roi_mask
    
    x1, y1, x2, y2 = bb
    H, W = roi_mask.shape
    bw, bh = x2 - x1 + 1, y2 - y1 + 1
    pad_x, pad_y = int(bw * pad_ratio), int(bh * pad_ratio)
    
    X1, Y1 = max(0, x1 - pad_x), max(0, y1 - pad_y)
    X2, Y2 = min(W-1, x2 + pad_x), min(H-1, y2 + pad_y)
    
    clipped = np.zeros_like(roi_mask, dtype=np.uint8)
    clipped[Y1:Y2+1, X1:X2+1] = roi_mask[Y1:Y2+1, X1:X2+1]
    return clipped

def border_touch_ratio(mask01):
    mask = (mask01 > 0).astype(np.uint8)
    H, W = mask.shape
    border = np.zeros_like(mask)
    border[0, :] = border[-1, :] = border[:, 0] = border[:, -1] = 1
    return float((mask & border).sum() / (mask.sum() + 1e-9))

def score_roi(labels, lesion_mask, chosen):
    if isinstance(chosen, int):
        chosen = [chosen]
    
    roi = np.zeros_like(labels, dtype=np.uint8)
    for kk in chosen:
        roi |= (labels == kk).astype(np.uint8)
    
    lesion_area = int(lesion_mask.sum())
    roi_area = int(roi.sum())
    inter = int((roi & lesion_mask).sum())
    
    if lesion_area == 0 or roi_area == 0 or inter == 0:
        return {"chosen": chosen, "f1": 0.0, "precision": 0.0, "recall": 0.0}
    
    precision = inter / (roi_area + 1e-9)
    recall = inter / (lesion_area + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)
    
    btr = border_touch_ratio(roi)
    f1_adj = f1 * (1.0 - 0.35 * min(1.0, btr))
    
    return {
        "chosen": chosen,
        "precision": float(precision),
        "recall": float(recall),
        "f1": float(f1_adj),
        "border_touch": float(btr)
    }

def pick_best_roi_clusters(labels, lesion_mask):
    k = int(labels.max()) + 1
    best = None
    
    for kk in range(k):
        info = score_roi(labels, lesion_mask, kk)
        if best is None or info["f1"] > best["f1"]:
            best = info
    
    for a in range(k):
        for b in range(a + 1, k):
            info = score_roi(labels, lesion_mask, [a, b])
            if info["f1"] > best["f1"]:
                best = info
    
    return best

def segment_guided_kmeans(bgr, txt_path, pad_ratio=PAD_RATIO):
    labels = kmeans_labels_lab(bgr, k=KMEANS_K)
    lesion_mask = lesion_mask_from_txt(labels.shape, txt_path)
    
    if int(lesion_mask.sum()) == 0:
        return labels, lesion_mask, None, None
    
    best = pick_best_roi_clusters(labels, lesion_mask)
    if best is None or best["f1"] <= 0:
        return labels, lesion_mask, None, None
    
    roi_mask = np.zeros_like(labels, dtype=np.uint8)
    for kk in best["chosen"]:
        roi_mask |= (labels == kk).astype(np.uint8)
    
    roi_mask = refine_roi_mask(roi_mask)
    roi_mask = clip_roi_to_context(roi_mask, lesion_mask, pad_ratio)
    roi_mask = refine_roi_mask(roi_mask)
    
    return labels, lesion_mask, roi_mask, best

Annotation txt files found: 545


# 4. Visualization & Save Samples

In [ ]:
def bgr_to_rgb(bgr):
    return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

def overlay_mask(bgr, mask, color=(0, 0, 255), alpha=0.35):
    overlay = bgr.copy()
    overlay[mask == 1] = (alpha * np.array(color, dtype=np.uint8) + 
                          (1 - alpha) * overlay[mask == 1]).astype(np.uint8)
    return overlay

def compute_prf(roi_mask, lesion_mask):
    roi = (roi_mask > 0).astype(np.uint8)
    les = (lesion_mask > 0).astype(np.uint8)
    inter = int((roi & les).sum())
    roi_area = int(roi.sum())
    les_area = int(les.sum())
    
    if roi_area == 0 or les_area == 0 or inter == 0:
        return 0.0, 0.0, 0.0
    
    precision = inter / (roi_area + 1e-9)
    recall = inter / (les_area + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)
    return float(precision), float(recall), float(f1)

SAMPLE_SIZE = 12
sample_df = df.sample(min(SAMPLE_SIZE, len(df)), random_state=RANDOM_STATE)

saved = 0
skipped_no_annot = 0

for i, r in enumerate(sample_df.to_dict("records"), start=1):
    img_path = r.get("orig_path", r.get("path"))
    bgr = read_bgr_300(img_path)
    if bgr is None:
        continue
    
    txt = find_annotation_txt(img_path)
    if txt is None:
        skipped_no_annot += 1
        continue
    
    labels, lesion_mask, roi_mask, best = segment_guided_kmeans(bgr, txt)
    if roi_mask is None:
        continue
    
    label_vis = (labels.astype(np.float32) / max(1, labels.max()) * 255).astype(np.uint8)
    label_vis = cv2.applyColorMap(label_vis, cv2.COLORMAP_JET)
    
    over_lesion = overlay_mask(bgr, lesion_mask, color=(0, 255, 255), alpha=0.35)
    over_roi = overlay_mask(bgr, roi_mask, color=(0, 0, 255), alpha=0.35)
    
    fig = plt.figure(figsize=(16, 4))
    
    ax1 = fig.add_subplot(1, 4, 1)
    ax1.imshow(bgr_to_rgb(bgr))
    ax1.set_title(f"Original ({r['class']})")
    ax1.axis("off")
    
    ax2 = fig.add_subplot(1, 4, 2)
    ax2.imshow(bgr_to_rgb(label_vis))
    ax2.set_title(f"K-Means (k={KMEANS_K}, LAB)")
    ax2.axis("off")
    
    ax3 = fig.add_subplot(1, 4, 3)
    ax3.imshow(bgr_to_rgb(over_lesion))
    ax3.set_title("YOLO Annotations")
    ax3.axis("off")
    
    ax4 = fig.add_subplot(1, 4, 4)
    ax4.imshow(bgr_to_rgb(over_roi))
    P, R, F = compute_prf(roi_mask, lesion_mask)
    ax4.set_title(f"ROI Clusters={best['chosen']}\nF1={F:.3f} (P={P:.3f}, R={R:.3f})")
    ax4.axis("off")
    
    out_img = SAMPLE_DIR / f"seg_{saved+1:02d}_{r['class']}.png"
    fig.savefig(out_img, bbox_inches="tight", dpi=100)
    plt.close(fig)
    saved += 1

print(f"\n✓ Saved {saved} segmentation samples to {SAMPLE_DIR}")
print(f"  Skipped (no annotation): {skipped_no_annot}")

c:\Users\User\.conda\envs\pengenalan-pola\Lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


Saved samples: 7
Skipped (no annotation found): 5
Output folder: E:\Kuliah\Pengenalan Pola\addressing_agricultural_challenges\outputs\samples\segmentation_guided
